### Running ANN  using GPU

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [6]:
torch.cuda.memory_cached()

C:\Users\Tanlocal\anaconda3\envs\torch1\Lib\site-packages\torch\cuda\memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


0

In [7]:
torch.cuda.memory_allocated()

0

In [8]:
var1=torch.FloatTensor([1.0,2.0,3.0]).cuda()

In [9]:
var1

tensor([1., 2., 3.], device='cuda:0')

In [10]:
var1.device

device(type='cuda', index=0)

In [11]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [13]:
import seaborn as sns

In [14]:
import numpy as np
df['Outcome']=np.where(df['Outcome']==1,"Diabetic","No Diabetic")

In [15]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,No Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,No Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic


In [19]:
#sns.pairplot(df,hue="Outcome")

In [20]:
X=df.drop('Outcome',axis=1).values### independent features
y=df['Outcome'].values###dependent features

In [21]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [22]:
y_train

array(['Diabetic', 'No Diabetic', 'No Diabetic', 'No Diabetic',
       'Diabetic', 'Diabetic', 'Diabetic', 'Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'No Diabetic',
       'No Diabetic', 'Diabetic', 'Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'No Diabetic',
       'No Diabetic', 'Diabetic', 'Diabetic', 'No Diabetic', 'Diabetic',
       'Diabetic', 'No Diabetic', 'Diabetic', 'Diabetic', 'Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'Diabetic', 'Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'Diabetic',
       'No Diabetic', 'Diabetic', 'No Diabetic', 'No Diabetic',
       'No Diabetic', 'No Diabetic', 'No Diabetic', 'Diabetic',
       'Diabe

In [23]:
#### Libraries From Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
##### Creating Tensors
X_train=torch.FloatTensor(X_train).cuda()
X_test=torch.FloatTensor(X_test).cuda()
y_train=torch.LongTensor(y_train).cuda()
y_test=torch.LongTensor(y_test).cuda()


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [25]:
X_train.device

device(type='cuda', index=0)

In [26]:
#### Creating Modelwith Pytorch

class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [27]:
####instantiate my ANN_model
torch.manual_seed(20)
model=ANN_Model()

In [28]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=2, bias=True)
)>

In [29]:
for i in model.parameters():
    print(i.is_cuda)

False
False
False
False
False
False


In [30]:
model=model.cuda()

In [31]:
###Backward Propogation-- Define the loss_function,define the optimizer
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [32]:
import time
start_time=time.time()
epochs=10000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(time.time()-start_time)

TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not numpy.ndarray

In [ ]:
### plot the loss function
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(epochs),final_losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
#### Prediction In X_test data
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

In [ ]:
#### Save the model
torch.save(model,'diabetes.pt')

In [ ]:
#### Save And Load the model
model=torch.load('diabetes.pt')

In [ ]:
model.eval()

In [ ]:
### Predcition of new data point
list(df.iloc[0,:-1])

In [ ]:
#### New Data
lst1=[6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

In [ ]:
new_data=torch.tensor(lst1)

In [ ]:
#### Predict new data using Pytorch
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())